In [408]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from datetime import datetime
from tensorflow import keras
from keras.layers import Dense
from sklearn import metrics
from tensorflow.keras import layers
from tensorflow.python.keras.utils import np_utils
from sklearn.model_selection import KFold
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import roc_curve, auc
from sklearn.pipeline import Pipeline
# multi-class classification with Keras
from keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
# from keras.wrappers.scikit_learn import KerasClassifier
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
print(tf.__version__)

2.15.0


In [409]:
# Read in data as pandas dataframe and display first 5 rows
data = pd.read_csv('diabetesmerge.csv')
data.head(100)

,Glucose,BloodPressure,Insulin,Age,Outcome
0,89,66,94,21,0
1,137,40,168,33,2
2,78,50,88,26,0
3,197,70,543,53,2
4,189,60,846,59,2
...,...,...,...,...,...
95,109,64,99,26,1
96,148,60,318,29,2
97,99,70,44,27,0
98,103,72,190,55,1


In [410]:
#One hot encoding
labels = data['Outcome']
# encode class values as integers
encoder = LabelEncoder()
 

In [411]:

encoder.fit(labels)
encoded_Y = encoder.transform(labels)

In [412]:

# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = to_categorical(encoded_Y)

In [413]:
dummy_y

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)

In [414]:
# Remove the labels from the features
# axis 1 refers to the columns
data= data.drop('Outcome', axis = 1)

# Saving feature names for later use
data_list = list(data.columns)

# Convert to numpy array
data = np.array(data)

In [415]:
dummy_y.shape

(728, 3)

In [416]:
# Split the data into training and testing sets
train_features, val_test_features, train_labels, val_test_labels = train_test_split(data, dummy_y, test_size = 0.3,
                                                                           shuffle=True, random_state = 42)
train_features,train_labels = SMOTE().fit_resample(train_features, train_labels)
X_val, X_test, Y_val, Y_test = train_test_split(val_test_features, val_test_labels, test_size=0.5, shuffle=True, random_state=42)
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', Y_test.shape)
print('Validation Features Shape:', X_val.shape)
print('Validation Label Shape:', Y_val.shape)

Training Features Shape: (606, 4)
Training Labels Shape: (606, 3)
Testing Features Shape: (110, 4)
Testing Labels Shape: (110, 3)
Validation Features Shape: (109, 4)
Validation Label Shape: (109, 3)


In [417]:
# # Split the data into training and testing sets
# train_features, val_test_features, train_labels, val_test_labels = train_test_split(data, dummy_y, test_size = 0.3,
#                                                                            shuffle=True, random_state = 42)
# train_features_smote,train_labels_smote = SMOTE().fit_resample(train_features, train_labels)
# X_val, X_test, Y_val, Y_test = train_test_split(val_test_features, val_test_labels, test_size=0.5, shuffle=True, random_state=42)
# print('Training Features Shape:', train_features_smote.shape)
# print('Training Labels Shape:', train_labels_smote.shape)
# print('Testing Features Shape:', X_test.shape)
# print('Testing Labels Shape:', Y_test.shape)
# print('Validation Features Shape:', X_val.shape)
# print('Validation Label Shape:', Y_val.shape)

In [418]:
df = pd.DataFrame(train_labels, columns=['Category1', 'Category2', 'Category3'])

# 使用value_counts方法统计每个唯一行向量的数量
counts = df.apply(tuple, axis=1).value_counts()

# 输出每个唯一行向量及其数量
for index, count in counts.items():
    print(f"Category: {list(index)}, Count: {count}")

Category: [0, 0, 1], Count: 202
Category: [0, 1, 0], Count: 202
Category: [1, 0, 0], Count: 202


In [419]:
# df = pd.DataFrame(train_labels_smote, columns=['normal', 'prediabetes', 'diabetes'])

# # 使用value_counts方法统计每个唯一行向量的数量
# counts = df.apply(tuple, axis=1).value_counts()

# # 输出每个唯一行向量及其数量
# for index, count in counts.items():
#     print(f"Category: {list(index)}, Count: {count}")

In [420]:
normalizer = preprocessing.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))

print(normalizer.mean.numpy())

first = np.array(train_features[:1])

with np.printoptions(precision=2, suppress=True):
  print('First example:', first)
  print()
  print('Normalized:', normalizer1(first).numpy())

[[116.70957   71.26402   74.5297    32.570957]]
First example: [[147  75   0  28]]

Normalized: [[ 0.83  0.24 -0.71 -0.47]]


In [421]:
# normalizer2 = preprocessing.Normalization(axis=-1)
# normalizer2.adapt(np.array(train_features_smote))

# print(normalizer2.mean.numpy())

# first = np.array(train_features[:1])

# with np.printoptions(precision=2, suppress=True):
#   print('First example:', first)
#   print()
#   print('Normalized:', normalizer2(first).numpy())

In [422]:
class CustomReLU(layers.Layer):
    def __init__(self):
        super(CustomReLU, self).__init__()

    def call(self, inputs):
        return tf.maximum(0.0, inputs)

In [441]:
class MetaAconC(tf.keras.layers.Layer):
    def __init__(self, r=16):
        super(MetaAconC, self).__init__()
        self.fc1 = tf.keras.layers.Dense(r, use_bias=False)
        self.fc2 = tf.keras.layers.Dense(1, use_bias=False)

        self.p1 = tf.Variable(tf.random.normal((1, 1)))
        self.p2 = tf.Variable(tf.random.normal((1, 1)))

        self.sigmoid = tf.keras.layers.Activation('sigmoid')

    def call(self, x, **kwargs):
        x_mean = tf.reduce_mean(x, axis=1, keepdims=True)
        beta = self.sigmoid(self.fc2(self.fc1(x_mean)))
        return (self.p1 * x - self.p2 * x) * self.sigmoid(beta * (self.p1 * x - self.p2 * x)) + self.p2 * x

In [442]:
class PReLU(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(PReLU, self).__init__(**kwargs)
        self.alpha = None

    def build(self, input_shape):
        # 为每个输入通道创建一个可训练的参数
        self.alpha = self.add_weight(name='alpha',
                                     shape=(input_shape[-1],),
                                     initializer='zeros',
                                     trainable=True)

    def call(self, inputs):
        # TensorFlow中的PReLU实现
        return tf.maximum(self.alpha * inputs, inputs)

In [443]:
def custom_loss_with_l2(y_true, y_pred):
    l2_loss = tf.add_n([tf.nn.l2_loss(v) for v in model1.trainable_variables if 'kernel' in v.name])
    cross_entropy_loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_true, y_pred))
    total_loss = cross_entropy_loss + 0.01 * l2_loss  # 设置L2正则化系数为0.01
    return total_loss

In [444]:
import tensorflow as tf

class Swish(tf.keras.layers.Layer):
    def __init__(self):
        super(Swish, self).__init__()

    def call(self, inputs):
        return tf.keras.activations.swish(inputs)

main model designed

In [445]:
class Atten_model(tf.keras.layers.Layer):
    def __init__(self, in_dim, out_dim, activation):
        super(Atten_model, self).__init__()
        self.k = tf.keras.layers.Dense(in_dim)
        self.q = tf.keras.layers.Dense(in_dim)
        self.v = tf.keras.layers.Dense(in_dim)
        self.proj = tf.keras.layers.Dense(out_dim)
        self.proj_drop = tf.keras.layers.Dropout(0.6)
        self.GELU = activation()
    def call(self, x):
        # k q v 均将x从in_dim转变为out_dim，特征拓展、特征对应一个权重
        k = self.k(x)
        q = self.q(x)
        v = self.v(x)
        # 点乘计算注意力
        atten = tf.nn.softmax((k * q) / tf.math.sqrt(tf.cast(v.shape[1], dtype=tf.float32)), axis=1)
        # 特征值
        out = self.proj(self.GELU(atten * v))
        return out

In [480]:
#create model_2GDNN
def create_model1(data):
  model = keras.Sequential([
      data,
#       Atten_model(4, 8, MetaAconC),
#       layers.Dense(4),
#       MetaAconC(),
#       layers.Dense(8),
#       MetaAconC(),
#       layers.Dense(8),
#       MetaAconC(),
#       layers.Dense(32),
#       MetaAconC(),
#       layers.Dense(16),
#       MetaAconC(),
#       layers.Dense(8),
#      MetaAconC(),
      layers.Dense(8),
     MetaAconC(),
      layers.Dense(16),
      MetaAconC(),
      layers.Dense(3, activation="softmax")
  ])
  model.compile(loss=custom_loss_with_l2, optimizer='RMSprop', metrics=['accuracy'])
  return model

In [481]:
#create model_2GDNN
def create_model2(data):
  model = keras.Sequential([
      data,
      Atten_model(4, 4, MetaAconC),
#       layers.Dense(8),
      MetaAconC(),
#       layers.Dense(8),
#       MetaAconC(),
#       layers.Dense(32),
#       MetaAconC(),
#       layers.Dense(16),
#       MetaAconC(),
#       layers.Dense(8),
#      MetaAconC(),
      layers.Dense(8),
     MetaAconC(),
      layers.Dense(16),
      MetaAconC(),
      layers.Dense(3, activation="softmax")
  ])
  model.compile(loss=custom_loss_with_l2, optimizer='RMSprop', metrics=['accuracy'])
  return model

In [482]:
#create model_2GDNN
def create_model3(data):
  model = keras.Sequential([
      data,
      Atten_model(4, 8, CustomReLU),
      CustomReLU(),
#       layers.Dense(8),
#       MetaAconC(),
#       layers.Dense(8),
#       MetaAconC(),
#       layers.Dense(32),
#       MetaAconC(),
#       layers.Dense(16),
#       MetaAconC(),
#       layers.Dense(8),
#      MetaAconC(),
      layers.Dense(8),
     CustomReLU(),
      layers.Dense(16),
      CustomReLU(),
      layers.Dense(3, activation="softmax")
  ])
  model.compile(loss=custom_loss_with_l2, optimizer='RMSprop', metrics=['accuracy'])
  return model

In [469]:
normalizer

In [483]:
model1 = create_model1(normalizer)

In [471]:
model2 = create_model2(normalizer)

In [472]:
model3 = create_model3(normalizer)

In [484]:
model1.summary()
model2.summary()
model3.summary()

Model: "sequential_69"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_9 (Normaliza  (None, 4)                 9         
 tion)                                                           
                                                                 
 dense_837 (Dense)           (None, 8)                 40        
                                                                 
 meta_acon_c_196 (MetaAconC  (None, 8)                 34        
 )                                                               
                                                                 
 dense_840 (Dense)           (None, 16)                144       
                                                                 
 meta_acon_c_197 (MetaAconC  (None, 16)                34        
 )                                                               
                                                     

In [485]:
train_features.shape,train_labels.shape

((606, 4), (606, 3))

In [486]:
# # Define the Keras TensorBoard callback.
# logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
# mcp_save = ModelCheckpoint('output/bestmodel.tf', save_best_only=True, monitor='val_loss', mode='min')

history1=model1.fit(
    train_features, train_labels,
     validation_data=(X_val, Y_val),
    verbose=1,batch_size=32, epochs=200)


Epoch 1/200
19/19 [==============================] - 1s 16ms/step - loss: 1.2853 - accuracy: 0.3267 - val_loss: 1.2554 - val_accuracy: 0.3761
Epoch 2/200
19/19 [==============================] - 0s 4ms/step - loss: 1.2062 - accuracy: 0.4406 - val_loss: 1.2007 - val_accuracy: 0.4037
Epoch 3/200
19/19 [==============================] - 0s 4ms/step - loss: 1.1494 - accuracy: 0.4967 - val_loss: 1.1537 - val_accuracy: 0.4587
Epoch 4/200
19/19 [==============================] - 0s 4ms/step - loss: 1.0958 - accuracy: 0.5314 - val_loss: 1.1065 - val_accuracy: 0.5596
Epoch 5/200
19/19 [==============================] - 0s 4ms/step - loss: 1.0441 - accuracy: 0.5693 - val_loss: 1.0592 - val_accuracy: 0.6055
Epoch 6/200
19/19 [==============================] - 0s 4ms/step - loss: 0.9937 - accuracy: 0.5908 - val_loss: 1.0099 - val_accuracy: 0.6330
Epoch 7/200
19/19 [==============================] - 0s 4ms/step - loss: 0.9449 - accuracy: 0.6205 - val_loss: 0.9611 - val_accuracy: 0.6330
Epoch 8/200


Epoch 59/200
19/19 [==============================] - 0s 3ms/step - loss: 0.4419 - accuracy: 0.9406 - val_loss: 0.3495 - val_accuracy: 0.9083
Epoch 60/200
19/19 [==============================] - 0s 3ms/step - loss: 0.4377 - accuracy: 0.9340 - val_loss: 0.3476 - val_accuracy: 0.9083
Epoch 61/200
19/19 [==============================] - 0s 3ms/step - loss: 0.4342 - accuracy: 0.9373 - val_loss: 0.3460 - val_accuracy: 0.9083
Epoch 62/200
19/19 [==============================] - 0s 4ms/step - loss: 0.4310 - accuracy: 0.9340 - val_loss: 0.3443 - val_accuracy: 0.9083
Epoch 63/200
19/19 [==============================] - 0s 3ms/step - loss: 0.4276 - accuracy: 0.9373 - val_loss: 0.3421 - val_accuracy: 0.9083
Epoch 64/200
19/19 [==============================] - 0s 4ms/step - loss: 0.4232 - accuracy: 0.9373 - val_loss: 0.3416 - val_accuracy: 0.9083
Epoch 65/200
19/19 [==============================] - 0s 4ms/step - loss: 0.4200 - accuracy: 0.9340 - val_loss: 0.3386 - val_accuracy: 0.9083
Epoch 

19/19 [==============================] - 0s 4ms/step - loss: 0.2028 - accuracy: 0.9785 - val_loss: 0.2163 - val_accuracy: 0.9633
Epoch 117/200
19/19 [==============================] - 0s 4ms/step - loss: 0.2016 - accuracy: 0.9802 - val_loss: 0.2151 - val_accuracy: 0.9633
Epoch 118/200
19/19 [==============================] - 0s 3ms/step - loss: 0.2001 - accuracy: 0.9802 - val_loss: 0.2132 - val_accuracy: 0.9633
Epoch 119/200
19/19 [==============================] - 0s 4ms/step - loss: 0.1979 - accuracy: 0.9818 - val_loss: 0.2081 - val_accuracy: 0.9633
Epoch 120/200
19/19 [==============================] - 0s 3ms/step - loss: 0.1976 - accuracy: 0.9802 - val_loss: 0.2097 - val_accuracy: 0.9633
Epoch 121/200
19/19 [==============================] - 0s 3ms/step - loss: 0.1950 - accuracy: 0.9802 - val_loss: 0.2061 - val_accuracy: 0.9633
Epoch 122/200
19/19 [==============================] - 0s 3ms/step - loss: 0.1935 - accuracy: 0.9835 - val_loss: 0.2030 - val_accuracy: 0.9817
Epoch 123/200

19/19 [==============================] - 0s 4ms/step - loss: 0.1489 - accuracy: 0.9868 - val_loss: 0.1568 - val_accuracy: 0.9908
Epoch 174/200
19/19 [==============================] - 0s 4ms/step - loss: 0.1488 - accuracy: 0.9868 - val_loss: 0.1547 - val_accuracy: 0.9908
Epoch 175/200
19/19 [==============================] - 0s 4ms/step - loss: 0.1477 - accuracy: 0.9884 - val_loss: 0.1544 - val_accuracy: 0.9908
Epoch 176/200
19/19 [==============================] - 0s 4ms/step - loss: 0.1460 - accuracy: 0.9884 - val_loss: 0.1527 - val_accuracy: 0.9908
Epoch 177/200
19/19 [==============================] - 0s 4ms/step - loss: 0.1472 - accuracy: 0.9884 - val_loss: 0.1522 - val_accuracy: 0.9908
Epoch 178/200
19/19 [==============================] - 0s 4ms/step - loss: 0.1459 - accuracy: 0.9917 - val_loss: 0.1556 - val_accuracy: 0.9908
Epoch 179/200
19/19 [==============================] - 0s 3ms/step - loss: 0.1466 - accuracy: 0.9884 - val_loss: 0.1510 - val_accuracy: 0.9908
Epoch 180/200

In [476]:
history1=model2.fit(
    train_features, train_labels,
     validation_data=(X_val, Y_val),
    verbose=1,batch_size=32, epochs=200)

Epoch 1/200
19/19 [==============================] - 3s 22ms/step - loss: 1.1890 - accuracy: 0.4290 - val_loss: 1.1884 - val_accuracy: 0.4128
Epoch 2/200
19/19 [==============================] - 0s 4ms/step - loss: 1.1780 - accuracy: 0.4538 - val_loss: 1.1830 - val_accuracy: 0.3945
Epoch 3/200
19/19 [==============================] - 0s 4ms/step - loss: 1.1628 - accuracy: 0.4769 - val_loss: 1.1778 - val_accuracy: 0.3761
Epoch 4/200
19/19 [==============================] - 0s 4ms/step - loss: 1.1424 - accuracy: 0.4884 - val_loss: 1.1728 - val_accuracy: 0.3670
Epoch 5/200
19/19 [==============================] - 0s 4ms/step - loss: 1.1193 - accuracy: 0.5050 - val_loss: 1.1652 - val_accuracy: 0.3578
Epoch 6/200
19/19 [==============================] - 0s 4ms/step - loss: 1.0930 - accuracy: 0.5297 - val_loss: 1.1531 - val_accuracy: 0.4037
Epoch 7/200
19/19 [==============================] - 0s 4ms/step - loss: 1.0650 - accuracy: 0.5363 - val_loss: 1.1363 - val_accuracy: 0.4220
Epoch 8/200


Epoch 59/200
19/19 [==============================] - 0s 4ms/step - loss: 0.2317 - accuracy: 0.9736 - val_loss: 0.1904 - val_accuracy: 0.9725
Epoch 60/200
19/19 [==============================] - 0s 4ms/step - loss: 0.2293 - accuracy: 0.9736 - val_loss: 0.1968 - val_accuracy: 0.9541
Epoch 61/200
19/19 [==============================] - 0s 4ms/step - loss: 0.2258 - accuracy: 0.9703 - val_loss: 0.1922 - val_accuracy: 0.9633
Epoch 62/200
19/19 [==============================] - 0s 4ms/step - loss: 0.2216 - accuracy: 0.9719 - val_loss: 0.1850 - val_accuracy: 0.9725
Epoch 63/200
19/19 [==============================] - 0s 4ms/step - loss: 0.2194 - accuracy: 0.9736 - val_loss: 0.1853 - val_accuracy: 0.9725
Epoch 64/200
19/19 [==============================] - 0s 4ms/step - loss: 0.2141 - accuracy: 0.9719 - val_loss: 0.1843 - val_accuracy: 0.9633
Epoch 65/200
19/19 [==============================] - 0s 4ms/step - loss: 0.2112 - accuracy: 0.9736 - val_loss: 0.1828 - val_accuracy: 0.9633
Epoch 

19/19 [==============================] - 0s 4ms/step - loss: 0.1311 - accuracy: 0.9884 - val_loss: 0.1236 - val_accuracy: 1.0000
Epoch 117/200
19/19 [==============================] - 0s 4ms/step - loss: 0.1319 - accuracy: 0.9851 - val_loss: 0.1297 - val_accuracy: 0.9908
Epoch 118/200
19/19 [==============================] - 0s 4ms/step - loss: 0.1283 - accuracy: 0.9917 - val_loss: 0.1253 - val_accuracy: 0.9908
Epoch 119/200
19/19 [==============================] - 0s 4ms/step - loss: 0.1290 - accuracy: 0.9851 - val_loss: 0.1254 - val_accuracy: 0.9908
Epoch 120/200
19/19 [==============================] - 0s 4ms/step - loss: 0.1275 - accuracy: 0.9868 - val_loss: 0.1254 - val_accuracy: 1.0000
Epoch 121/200
19/19 [==============================] - 0s 4ms/step - loss: 0.1282 - accuracy: 0.9884 - val_loss: 0.1278 - val_accuracy: 0.9908
Epoch 122/200
19/19 [==============================] - 0s 4ms/step - loss: 0.1252 - accuracy: 0.9917 - val_loss: 0.1296 - val_accuracy: 0.9908
Epoch 123/200

19/19 [==============================] - 0s 5ms/step - loss: 0.1113 - accuracy: 0.9967 - val_loss: 0.1123 - val_accuracy: 0.9908
Epoch 174/200
19/19 [==============================] - 0s 4ms/step - loss: 0.1090 - accuracy: 0.9950 - val_loss: 0.1109 - val_accuracy: 1.0000
Epoch 175/200
19/19 [==============================] - 0s 4ms/step - loss: 0.1086 - accuracy: 1.0000 - val_loss: 0.1141 - val_accuracy: 1.0000
Epoch 176/200
19/19 [==============================] - 0s 4ms/step - loss: 0.1085 - accuracy: 0.9950 - val_loss: 0.1095 - val_accuracy: 1.0000
Epoch 177/200
19/19 [==============================] - 0s 4ms/step - loss: 0.1101 - accuracy: 0.9967 - val_loss: 0.1109 - val_accuracy: 1.0000
Epoch 178/200
19/19 [==============================] - 0s 4ms/step - loss: 0.1096 - accuracy: 0.9967 - val_loss: 0.1094 - val_accuracy: 1.0000
Epoch 179/200
19/19 [==============================] - 0s 4ms/step - loss: 0.1080 - accuracy: 0.9967 - val_loss: 0.1092 - val_accuracy: 1.0000
Epoch 180/200

In [477]:
history3=model3.fit(
    train_features, train_labels,
     validation_data=(X_val, Y_val),
    verbose=1,batch_size=32, epochs=200)

Epoch 1/200
19/19 [==============================] - 1s 16ms/step - loss: 1.1291 - accuracy: 0.5611 - val_loss: 1.0985 - val_accuracy: 0.5780
Epoch 2/200
19/19 [==============================] - 0s 3ms/step - loss: 1.0900 - accuracy: 0.5825 - val_loss: 1.0625 - val_accuracy: 0.6055
Epoch 3/200
19/19 [==============================] - 0s 4ms/step - loss: 1.0555 - accuracy: 0.5974 - val_loss: 1.0254 - val_accuracy: 0.6239
Epoch 4/200
19/19 [==============================] - 0s 4ms/step - loss: 1.0200 - accuracy: 0.6122 - val_loss: 0.9884 - val_accuracy: 0.6330
Epoch 5/200
19/19 [==============================] - 0s 3ms/step - loss: 0.9846 - accuracy: 0.6254 - val_loss: 0.9545 - val_accuracy: 0.6239
Epoch 6/200
19/19 [==============================] - 0s 3ms/step - loss: 0.9500 - accuracy: 0.6485 - val_loss: 0.9232 - val_accuracy: 0.6147
Epoch 7/200
19/19 [==============================] - 0s 3ms/step - loss: 0.9160 - accuracy: 0.6601 - val_loss: 0.8937 - val_accuracy: 0.6422
Epoch 8/200


Epoch 59/200
19/19 [==============================] - 0s 4ms/step - loss: 0.3007 - accuracy: 0.9439 - val_loss: 0.3875 - val_accuracy: 0.8991
Epoch 60/200
19/19 [==============================] - 0s 3ms/step - loss: 0.3025 - accuracy: 0.9439 - val_loss: 0.3676 - val_accuracy: 0.9083
Epoch 61/200
19/19 [==============================] - 0s 3ms/step - loss: 0.3001 - accuracy: 0.9422 - val_loss: 0.3457 - val_accuracy: 0.9083
Epoch 62/200
19/19 [==============================] - 0s 3ms/step - loss: 0.2977 - accuracy: 0.9439 - val_loss: 0.3649 - val_accuracy: 0.9174
Epoch 63/200
19/19 [==============================] - 0s 3ms/step - loss: 0.2966 - accuracy: 0.9472 - val_loss: 0.3534 - val_accuracy: 0.9266
Epoch 64/200
19/19 [==============================] - 0s 3ms/step - loss: 0.2951 - accuracy: 0.9455 - val_loss: 0.3438 - val_accuracy: 0.9174
Epoch 65/200
19/19 [==============================] - 0s 3ms/step - loss: 0.2926 - accuracy: 0.9472 - val_loss: 0.3344 - val_accuracy: 0.9266
Epoch 

19/19 [==============================] - 0s 4ms/step - loss: 0.2331 - accuracy: 0.9620 - val_loss: 0.3016 - val_accuracy: 0.9450
Epoch 117/200
19/19 [==============================] - 0s 4ms/step - loss: 0.2329 - accuracy: 0.9587 - val_loss: 0.2872 - val_accuracy: 0.9358
Epoch 118/200
19/19 [==============================] - 0s 4ms/step - loss: 0.2312 - accuracy: 0.9637 - val_loss: 0.3029 - val_accuracy: 0.9358
Epoch 119/200
19/19 [==============================] - 0s 4ms/step - loss: 0.2323 - accuracy: 0.9620 - val_loss: 0.3067 - val_accuracy: 0.9358
Epoch 120/200
19/19 [==============================] - 0s 4ms/step - loss: 0.2296 - accuracy: 0.9637 - val_loss: 0.3037 - val_accuracy: 0.9358
Epoch 121/200
19/19 [==============================] - 0s 4ms/step - loss: 0.2284 - accuracy: 0.9670 - val_loss: 0.3182 - val_accuracy: 0.9358
Epoch 122/200
19/19 [==============================] - 0s 4ms/step - loss: 0.2272 - accuracy: 0.9620 - val_loss: 0.2874 - val_accuracy: 0.9358
Epoch 123/200

19/19 [==============================] - 0s 4ms/step - loss: 0.1991 - accuracy: 0.9703 - val_loss: 0.3188 - val_accuracy: 0.9450
Epoch 174/200
19/19 [==============================] - 0s 3ms/step - loss: 0.1984 - accuracy: 0.9686 - val_loss: 0.3074 - val_accuracy: 0.9450
Epoch 175/200
19/19 [==============================] - 0s 4ms/step - loss: 0.2008 - accuracy: 0.9703 - val_loss: 0.3029 - val_accuracy: 0.9450
Epoch 176/200
19/19 [==============================] - 0s 3ms/step - loss: 0.1985 - accuracy: 0.9736 - val_loss: 0.3285 - val_accuracy: 0.9358
Epoch 177/200
19/19 [==============================] - 0s 3ms/step - loss: 0.1983 - accuracy: 0.9653 - val_loss: 0.3031 - val_accuracy: 0.9541
Epoch 178/200
19/19 [==============================] - 0s 4ms/step - loss: 0.1997 - accuracy: 0.9719 - val_loss: 0.3376 - val_accuracy: 0.9358
Epoch 179/200
19/19 [==============================] - 0s 3ms/step - loss: 0.1949 - accuracy: 0.9752 - val_loss: 0.3510 - val_accuracy: 0.9450
Epoch 180/200

In [487]:
# Compare Prediction 
print("Train acc: " , model1.evaluate(train_features, train_labels))
print("Test acc: ", model1.evaluate(X_test, Y_test))
print("Train acc: " , model2.evaluate(train_features, train_labels))
print("Test acc: ", model2.evaluate(X_test, Y_test))

19/19 [==============================] - 0s 2ms/step - loss: 0.1321 - accuracy: 0.9917
Train acc:  [0.13209675252437592, 0.9917491674423218]
4/4 [==============================] - 0s 3ms/step - loss: 0.1799 - accuracy: 0.9545
Test acc:  [0.17985223233699799, 0.9545454382896423]
19/19 [==============================] - 0s 2ms/step - loss: 0.1036 - accuracy: 1.0000
Train acc:  [0.10364677011966705, 1.0]
4/4 [==============================] - 0s 3ms/step - loss: 0.1452 - accuracy: 0.9818
Test acc:  [0.14517098665237427, 0.9818181991577148]


In [488]:
print("Train acc: " , model3.evaluate(train_features, train_labels))
print("Test acc: ", model3.evaluate(X_test, Y_test))

19/19 [==============================] - 0s 2ms/step - loss: 0.1845 - accuracy: 0.9752
Train acc:  [0.18447068333625793, 0.9752475023269653]
4/4 [==============================] - 0s 3ms/step - loss: 0.8634 - accuracy: 0.9182
Test acc:  [0.8633522987365723, 0.918181836605072]
